In [25]:
%load_ext bigdata

The bigdata extension is already loaded. To reload it, use:
  %reload_ext bigdata


In [26]:
%pig_start

In [27]:
%timeout 300

In [28]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-m-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [29]:
!hadoop fs -put *.csv .
!hadoop fs -ls

put: `data.csv': File exists
Found 3 items
-rw-r--r--   1 root supergroup        615 2020-01-18 23:22 data.csv
-rw-r--r--   1 root supergroup       1741 2020-01-18 22:51 data.tsv
-rw-r--r--   1 root supergroup    2271958 2020-01-18 22:32 truck_event_text_partition.csv


In [30]:
!rm -rf input
!rm -rf output
!mkdir input
!mkdir output

In [31]:
%%writefile script.pig

-- crea la carpeta input in el HDFS
-- fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put input/ .

-- carga de datos
--Punto 17

u = LOAD 'data.csv' USING PigStorage(',')
    AS (f1:INT, f2:CHARARRAY, f3:CHARARRAY, f4:CHARARRAY, f5:CHARARRAY, f6:INT);
    
y = FILTER u BY (f5 matches '.*bl.*');

b = FOREACH y GENERATE CONCAT(f2,',', f5);

DUMP b;

-- escribe el archivo de salida
STORE b INTO 'output';

-- copia los warchivos del HDFS al sistema local
fs -get output/ .

Overwriting script.pig


In [32]:
!pig -execute 'run script.pig'

2020-01-19 03:23:40,076 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2020-01-19 03:23:43,246 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2020-01-19 03:23:43,527 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2020-01-19 03:23:43,533 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2020-01-19 03:23:43,553 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.replication
2020-01-19 03:23:44,220 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, us

In [33]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2020-01-19 03:24 output/_SUCCESS
-rw-r--r--   1 root supergroup         56 2020-01-19 03:24 output/part-m-00000


In [34]:
!hadoop fs -cat output/part-m-00000

Roth,black
Zoe,blue
Driscoll,blue
Kylan,black
Hope,blue


In [35]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rm output2/*
!hadoop fs -rmdir input output
!hadoop fs -rmdir input output2

rm: `input/*': No such file or directory
Deleted output/_SUCCESS
Deleted output/part-m-00000
rm: `output2/*': No such file or directory
rmdir: `input': No such file or directory
rmdir: `output2': No such file or directory


In [36]:
!rm -rf input
!rm -rf output